In [7]:
# TESTS THAT INTEGRATION WITH R WORKS
# https://github.com/rpy2/rpy2/issues/1018
# C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\rpy2

from rpy2 import situation
import os

try:
    import rpy2.robjects as robjects
except OSError as e:
    try:
        import os
        import platform
        
        r_home = situation.r_home_from_registry()
        r_bin = r_home + '\\bin\\x64\\'
        os.environ['R_HOME'] = r_home
        os.environ['PATH'] =  r_bin + ";" + os.environ['PATH']
        os.add_dll_directory(r_bin)

        print('R_HOME', r_home)
        import rpy2.robjects as robjects
    except OSError:
        raise(e)

print("R Worked!")

R Worked!


In [8]:
import pandas as pd

def compute_theta(filepath):

    # Read the content of the R script file
    r_script_file = "fit_irt.R"
    with open(r_script_file, 'r') as file:
        r_script = file.read()

    response_pattern_filepath =  f"{filepath}/aggregated/samples.csv"
    print("response_pattern_filepath", response_pattern_filepath)

    theta_file = f"{filepath}/aggregated/samples_with_theta.csv"
    print("theta_file", theta_file)

    if "2022" in response_pattern_filepath:
        YEAR = "2022"
    if "2021" in response_pattern_filepath:
        YEAR = "2021"
    if "2020" in response_pattern_filepath:
        YEAR = "2020"
        
    file_itens_prova = f"../microdados/microdados_enem_{YEAR}/DADOS/ITENS_PROVA_{YEAR}.csv"

    robjects.r.assign('response_pattern_filepath', f"{filepath}/aggregated/majority_sample.csv")
    robjects.r.assign('theta_file', f"{filepath}/aggregated/majority_sample_with_theta.csv")
    robjects.r.assign('file_itens_prova', file_itens_prova)
    robjects.r(r_script)

    robjects.r.assign('response_pattern_filepath', response_pattern_filepath)
    robjects.r.assign('theta_file', theta_file)
    robjects.r.assign('file_itens_prova', file_itens_prova)
    robjects.r(r_script)

    robjects.r.assign('response_pattern_filepath', f"{filepath}/aggregated/random_samples.csv")
    robjects.r.assign('theta_file', f"{filepath}/aggregated/random_samples_with_theta.csv")
    robjects.r.assign('file_itens_prova', file_itens_prova)
    robjects.r(r_script)

    thetas_df = pd.read_csv(f"{filepath}/aggregated/samples_with_theta.csv")
    thetas_df.sort_values(by=['IRT_SCORE', 'CTT_SCORE'], ascending=False).to_csv(f"{filepath}/aggregated/samples_with_theta.csv")
    
    thetas_random_df = pd.read_csv(f"{filepath}/aggregated/random_samples_with_theta.csv")
    thetas_random_df.sort_values(by=['IRT_SCORE', 'CTT_SCORE'], ascending=False).to_csv(f"{filepath}/aggregated/random_samples_with_theta.csv")
    print("thetas.csv finished! rows:", thetas_df.shape[0])

    #average_df = thetas_df.groupby('CTT_SCORE')['IRT_SCORE'].agg({'Value': ['mean', 'median', 'min', 'max']})

    average_df = thetas_df.groupby('CTT_SCORE')['IRT_SCORE'].mean().reset_index()
    average_random_df = thetas_random_df.groupby('CTT_SCORE')['IRT_SCORE'].mean().reset_index()
    
    return thetas_df, average_df, thetas_random_df, average_random_df

In [9]:
import os
import zipfile
import tempfile
import random
import string
from pathlib import Path

def compute_everything(source_directory, target_directory):
    # Get a list of all files in the specified directory
    file_list = os.listdir(source_directory)
    print(file_list)

    for file_name in file_list:
        # Check if the file is a zip file
        if file_name.startswith("exp_") and file_name.endswith(".zip"):
            print('file_name', file_name)
            zip_path = os.path.join(source_directory, file_name)

            # exp_CN_2022_mistral_simple-zero-shot_bits_4_count_7.zip
            (exp, exam, year, llm, prompt, b, bits, c, count) = file_name.split("_")
            print('exam', exam, 'year', year, 'prompt', prompt, 'bits', bits, 'count', count)

            Path(f"{target_directory}/{exam}").mkdir(parents=True, exist_ok=True)
            Path(f"{target_directory}/{exam}/{year}").mkdir(parents=True, exist_ok=True)
            Path(f"{target_directory}/{exam}/{year}/{llm}").mkdir(parents=True, exist_ok=True)
            Path(f"{target_directory}/{exam}/{year}/{llm}/{prompt}").mkdir(parents=True, exist_ok=True)

            full_target_directory = f"{target_directory}/{exam}/{year}/{llm}/{prompt}" 
            print('Will write to', full_target_directory)

            if os.path.exists(f"{full_target_directory}/aggregated/samples_with_theta.csv"):
                print(full_target_directory, ' exists, skipping.')
                continue
                
            # Create a ZipFile object
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # Extract the contents to the temporary directory
                zip_ref.extractall(full_target_directory)
                print(f"Files extracted to: {full_target_directory}")
                
                thetas_df, average_df, thetas_random_df, average_random_df = compute_theta(full_target_directory)
                average_df.to_csv(full_target_directory + "/aggregated/average_theta_by_score_sample.csv")
                average_random_df.to_csv(full_target_directory + "/aggregated/average_theta_by_score_random_sample.csv")

# Example usage:
source_directory_path = "C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL"
target_directory_path = "C:/Users/pedro/Downloads/TRI/test_responses_llms/EXP"
compute_everything(source_directory_path, target_directory_path)

print("FINISHED.")


['exp_CH_2020_llama2_simple-zero-shot_bits_4_count_1.zip', 'exp_CH_2020_mistral_paper-nunes-2023-zero-shot_bits_4_count_10.zip', 'exp_CH_2020_mistral_simple-zero-shot_bits_4_count_10.zip', 'exp_CH_2021_llama2_simple-zero-shot_bits_4_count_1.zip', 'exp_CH_2021_mistral_paper-nunes-2023-zero-shot_bits_4_count_10.zip', 'exp_CH_2021_mistral_simple-zero-shot_bits_4_count_10.zip', 'exp_CH_2022_llama2_simple-zero-shot_bits_4_count_1.zip', 'exp_CH_2022_mistral_paper-nunes-2023-zero-shot_bits_4_count_10.zip', 'exp_CH_2022_mistral_simple-zero-shot_bits_4_count_24.zip', 'exp_CN_2022_mistral_simple-zero-shot_bits_4_count_27.zip', 'exp_MT_2020_mistral_simple-zero-shot_bits_4_count_1.zip', 'exp_MT_2021_mistral_simple-zero-shot_bits_4_count_1.zip', 'exp_MT_2022_mistral_simple-zero-shot_bits_4_count_24.zip']
file_name exp_CH_2020_llama2_simple-zero-shot_bits_4_count_1.zip
exam CH year 2020 prompt simple-zero-shot bits 4 count 1.zip
Will write to C:/Users/pedro/Downloads/TRI/test_responses_llms/EXP/CH/2

In [23]:
df = pd.read_parquet("C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL/GABRIEL/enem-experiments-results-processed.parquet")

#df.sort_values(by='CTT_SCORE', ascending=False)[['ENEM_EXAM','MODEL_NAME','MODEL_SIZE','LANGUAGE','CTT_SCORE']]

#df.to_csv("C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL/enem-experiments-results-processed.csv")

df.head()

,MODEL_NAME,MODEL_SIZE,TEMPERATURE,SYSTEM_PROMPT_TYPE,ENEM_EXAM,ENEM_EXAM_TYPE,QUESTION_ORDER,LANGUAGE,NUMBER_OPTIONS,SEED,CTT_SCORE,TX_RESPOSTAS,TX_GABARITO,RESPONSE_PATTERN,TOTAL_RUN_TIME_SEC,AVG_RUN_TIME_PER_ITEM_SEC
0,gpt-3.5-turbo,None,0.6,few-shot,ENEM_2022_CH_CO_PROVA_1062,default,original,en,5,2724839799,39,CBECABBCDBACDCEDAADDBBDDAECDEBAECBDBAEEDADEAB,CBECABBCDEACDCEDAAADBCDDAECBABAECBEBAEEDADEAB,111111111011111111011011111001111101111111111,27.535338,0.611896
1,gpt-3.5-turbo,None,0.6,few-shot,ENEM_2022_CH_CO_PROVA_1062,default,original,pt-br,5,2724839799,40,CBECABBCDBACDCEDAADDBBDDAECCCBAECBEBAEEDADEAB,CBECABBCDEACDCEDAAADBCDDAECBABAECBEBAEEDADEAB,111111111011111111011011111001111111111111111,30.631764,0.680706
2,gpt-3.5-turbo,None,0.6,few-shot,ENEM_2022_CN_CO_PROVA_1092,default,original,en,5,2724839799,25,DCEBDCECECBEBEEDCACCDDCCCDBCCEABBABBEDBEDADAB,DDECDBEACCAEBEAEBBCCDDCBDDACBEACEABCEABEDADBA,101010100101110000111110010101100110101111100,32.228868,0.716197
3,gpt-3.5-turbo,None,0.6,few-shot,ENEM_2022_CN_CO_PROVA_1092,default,original,pt-br,5,2724839799,28,DBECDBEACCAEBEDBDACCEDCCDDECDEEDBACDEBBEDADDD,DDECDBEACCAEBEAEBBCCDDCBDDACBEACEABCEABEDADBA,101111111111110000110110110101000100101111100,31.431348,0.698474
4,gpt-3.5-turbo,None,0.6,few-shot,ENEM_2022_LC_CO_PROVA_1072,default,original,en,5,2724839799,37,BDECDAADCECCCDCCDBEEBBCEEBECACACCEDEDEEDCADBDA...,BDECDAADCECCCDCCEBACBBAAAEECABABEECEBEEDCADBDB...,1111111111111111010011000011101001010111111110...,40.699889,0.813998


In [28]:
df = pd.read_csv("C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL/GABRIEL/enem-experiments-results-processed.csv")
theta_df = pd.read_csv("C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL/GABRIEL/thetas-enem-experiments-results-processed.csv")

result_df = pd.concat([df, theta_df], axis=1, verify_integrity=False)
result_df = result_df.T.drop_duplicates().T

result_df.to_csv("C:/Users/pedro/Downloads/TRI/test_responses_llms/ZIPS/ALL/GABRIEL/experiments-with-irt.csv")

In [59]:
# 1062 CH
# 1082 MT
# 1092 CN


result_df[result_df['CO_PROVA'] == 1092][['ENEM_EXAM', 'MODEL_NAME', 'LANGUAGE', 'MODEL_SIZE','CTT_SCORE','IRT_SCORE', 'MEAN_CORRECT_B', 'MEAN_INCORRECT_B ']].sort_values(by='CTT_SCORE', ascending=False)



,ENEM_EXAM,MODEL_NAME,LANGUAGE,MODEL_SIZE,CTT_SCORE,IRT_SCORE,MEAN_CORRECT_B,MEAN_INCORRECT_B
3,ENEM_2022_CN_CO_PROVA_1092,gpt-3.5-turbo,pt-br,NaN,28,1.578906,1.413377,1.814693
2,ENEM_2022_CN_CO_PROVA_1092,gpt-3.5-turbo,en,NaN,25,1.233587,1.450004,1.678711
27,ENEM_2022_CN_CO_PROVA_1092,mistral,pt-br,7b,23,1.026545,1.406742,1.687241
26,ENEM_2022_CN_CO_PROVA_1092,mistral,en,7b,22,1.193356,1.27827,1.836161
11,ENEM_2022_CN_CO_PROVA_1092,llama2,pt-br,13b,18,0.82883,1.304695,1.692172
18,ENEM_2022_CN_CO_PROVA_1092,llama2,en,7b,17,0.816756,1.385414,1.65601
10,ENEM_2022_CN_CO_PROVA_1092,llama2,en,13b,16,0.707576,1.304799,1.692112
19,ENEM_2022_CN_CO_PROVA_1092,llama2,pt-br,7b,13,0.044838,1.194885,1.665098
